In [ ]:
from huggingface_hub import login
login('YOUR LOGIN KEY')


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
from sentence_transformers import SentenceTransformer
import pinecone
import transformers
import torch
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

In [ ]:
from pinecone import Pinecone, ServerlessSpec

api_key = 'YOUR API KEY'  # Replace with your actual API key
pinecone_client = Pinecone(api_key=api_key)

# Create or connect to a specific index
index_name = 'vector-nitor'  # Replace with your desired index name
if index_name not in pinecone_client.list_indexes().names():
    pinecone_client.create_index(
        name=index_name,
        dimension=384,  # 384 is the dimension for 'all-MiniLM-L6-v2' embeddings
        metric='cosine',  # Choose an appropriate metric for similarity
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'  # Adjust the region as needed
        )
    )

index = pinecone_client.Index(index_name)


In [6]:
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
model_id = "meta-llama/Llama-3.2-3B-Instruct"
llama_pipeline =transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="cuda"
)


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [10]:
def generate_query_embedding(query):
    """Generate an embedding for the user query."""
    return embedding_model.encode(query).tolist()

def retrieve_relevant_chunks(query_embedding, top_k=5):
    """Retrieve top-k relevant chunks from Pinecone based on query embedding and extract only the text."""
    results = index.query(vector=query_embedding, top_k=top_k, include_metadata=True)
    # Extract only the 'text' field from the metadata of each match
    relevant_texts = [match['metadata']['text'] for match in results['matches'] if 'text' in match['metadata']]
    return relevant_texts


def construct_prompt(relevant_chunks, query):
    """Construct a prompt for LLaMA model based on retrieved chunks."""
    text = "\n\n".join(relevant_chunks)
    context= f"Context:\n{text}"
    prompt="Generate answer according to the given context for the query: "+ query
    return prompt, context


In [ ]:


def get_answer_from_llama(prompt,context):
    """Generate response using the LLaMA pipeline with formatted messages."""
    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": context},
    ]
    outputs = llama_pipeline(
        messages,
        max_new_tokens=256,
        )
    return outputs[0]['generated_text'][-1]['content']  # Clean up any leading/trailing whitespace

def respond_to_query(query):
    """End-to-end RAG pipeline to respond to user query."""
    query_embedding = generate_query_embedding(query)
    relevant_chunks = retrieve_relevant_chunks(query_embedding)
    prompt,context = construct_prompt(relevant_chunks, query)
    answer = get_answer_from_llama(prompt, context)
    return context,answer

# Example usage
# user_query = "What are the diagnostic criteria for confirming a case of Diabetes Mellitus?"
# context,response = respond_to_query(user_query)
# print("Generated Response:", response)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Generated Response: The diagnostic criteria for confirming a case of Diabetes Mellitus are:

1. Fasting plasma glucose (FPG) ≥ 126 mg/dl
2. Random plasma glucose (RPG) ≥ 200 mg/dl with symptoms of diabetes (polyuria, polydipsia, ketoacidosis, or unexplained weight loss)
3. Two-hour plasma glucose (2hPG) ≥ 200 mg/dl following a 75g glucose load or 1.75g/kg body weight in children

These criteria are used to confirm a diagnosis of diabetes mellitus, with the understanding that a single abnormal result may not be sufficient to make a definitive diagnosis. The diagnosis is typically confirmed by repeating the test, and the results must meet the above criteria to be considered a confirmed case of diabetes mellitus.


## Evaluation

In [23]:
response

"The key symptoms that indicate a possible case of Type 1 Diabetes in children and young adults are:\n\n1. Fever\n2. Pain abdomen\n3. Polyuria (excessive urination)\n4. Weight loss\n\nThese symptoms are often mistaken for an acute infective illness in children, but they can be a sign of Type 1 Diabetes. Type 1 Diabetes is an autoimmune condition that destroys the insulin-producing beta cells in the pancreas, leading to a complete lack of insulin production and a requirement for insulin therapy.\n\nIt's essential to suspect and diagnose Type 1 Diabetes early, as it can lead to severe complications if left untreated. If you suspect a child or young adult has Type 1 Diabetes, it's crucial to seek medical attention and refer them to a higher-level facility for further evaluation and treatment."

In [31]:
gpt_res='''Diabetes Mellitus is diagnosed primarily through blood glucose tests. The criteria include:

Fasting Plasma Glucose (FPG): A blood test after at least 8 hours of fasting. A reading of ≥126 mg/dL (7.0 mmol/L) indicates diabetes.

Oral Glucose Tolerance Test (OGTT): Measures blood glucose two hours after consuming a glucose-rich drink. A reading of ≥200 mg/dL (11.1 mmol/L) confirms diabetes.

Random Plasma Glucose Test: Used when symptoms (e.g., polyuria, polydipsia) are present. A level of ≥200 mg/dL (11.1 mmol/L) indicates diabetes.

Hemoglobin A1c (HbA1c): Reflects average blood glucose over three months. An HbA1c level ≥6.5% confirms diabetes. For accuracy, HbA1c tests should be performed in a standardized lab.

If results fall within prediabetes ranges, further testing or lifestyle intervention may be recommended.'''

In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


def evaluation_matrics(response, context):
  bleu_score = sentence_bleu([context], response)
  response_vector = model.encode(response)
  context_vector = model.encode(context)
  response_vector = response_vector.reshape(1, -1)
  context_vector = context_vector.reshape(1, -1)
  similarity = cosine_similarity(response_vector, context_vector)
  print(bleu_score,similarity[0][0])
  return bleu_score, similarity[0][0]


BLEU score: 0.4358697266297498


In [ ]:
import gradio as gr

def run(query):
  context, answer = respond_to_query(query)
  print(answer,prompt)
  bleu, similarity = evaluation_matrics(answer, concat_result)
  print(bleu,similarity)
  return answer



demo = gr.Interface(
    fn=run,
    inputs=["text"],
    outputs=["text"],
)